In [ ]:
import pybullet as p
import time
import pybullet_data
from numpngw import write_apng
from IPython.display import Image
import numpy as np

p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())


p.configureDebugVisualizer(p.COV_ENABLE_GUI)
useFixedBase = True
flags = p.URDF_INITIALIZE_SAT_FEATURES

plane_pos = [0,0,-0.625]
plane = p.loadURDF("plane.urdf", plane_pos, flags = flags, useFixedBase=useFixedBase)
table_pos = [0,0,-0.625]
table = p.loadURDF("table/table.urdf", table_pos, flags = flags, useFixedBase=useFixedBase)
xarm = p.loadURDF("meshes/ur5.urdf", flags = flags, useFixedBase=useFixedBase)

table_pos = [0,1,-0.625]
table2 = p.loadURDF("table/table.urdf", table_pos, flags = flags, useFixedBase=useFixedBase)

sphereRadius = 0.25

p.createCollisionShape(p.GEOM_PLANE)
colBoxId = p.createCollisionShape(p.GEOM_BOX,halfExtents=[sphereRadius, sphereRadius, sphereRadius])

basePosition = [0,0.8,5]
baseOrientation = [0,0,0,1]
mass = 1
visualShapeId = -1

sphereUid = p.createMultiBody(mass, colBoxId, visualShapeId, basePosition,baseOrientation)

#set gravity in negative z direction
p.setGravity(0, 0, -10)
# p.setRealTimeSimulation(1)


fov, aspect, nearplane, farplane = 60, 1.0, 0.01, 100
projection_matrix = p.computeProjectionMatrixFOV(fov, aspect, nearplane, farplane)

def kuka_camera():
    # Center of mass position and orientation (of link-7)
    com_p, com_o, _, _, _, _ = p.getLinkState(xarm, 5, computeForwardKinematics=True)
    rot_matrix = p.getMatrixFromQuaternion(com_o)
    rot_matrix = np.array(rot_matrix).reshape(3, 3)
    # Initial vectors
    init_camera_vector = (0, 0, 1) # z-axis
    init_up_vector = (0, 1, 0) # y-axis
    # Rotated vectors
    camera_vector = rot_matrix.dot(init_camera_vector)
    up_vector = rot_matrix.dot(init_up_vector)
    view_matrix = p.computeViewMatrix(com_p, com_p + 0.1 * camera_vector, up_vector)
    img = p.getCameraImage(1000, 1000, view_matrix, projection_matrix)
    return img


while True:
    p.setRealTimeSimulation(1)
    kuka_camera()
    
p.disconnect()

##### 

In [1]:
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2


import numba
from numba import jit


import pybullet as p
import time
import pybullet_data
from numpngw import write_apng
from IPython.display import Image
import numpy as np

p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())


p.configureDebugVisualizer(p.COV_ENABLE_GUI)
useFixedBase = True
flags = p.URDF_INITIALIZE_SAT_FEATURES

plane_pos = [0,0,-0.625]
plane = p.loadURDF("plane.urdf", plane_pos, flags = flags, useFixedBase=useFixedBase)
table_pos = [0,0,-0.625]
table = p.loadURDF("table/table.urdf", table_pos, flags = flags, useFixedBase=useFixedBase)
xarm = p.loadURDF("meshes/untitiled.urdf", flags = flags, useFixedBase=useFixedBase)

table_pos = [0,1,-0.625]
table2 = p.loadURDF("table/table.urdf", table_pos, flags = flags, useFixedBase=useFixedBase)

sphereRadius = 0.25

p.createCollisionShape(p.GEOM_PLANE)
colBoxId = p.createCollisionShape(p.GEOM_BOX,halfExtents=[sphereRadius, sphereRadius, sphereRadius])

basePosition = [0,0.8,5]
baseOrientation = [0,0,0,1]
mass = 1
visualShapeId = -1

sphereUid = p.createMultiBody(mass, colBoxId, visualShapeId, basePosition,baseOrientation)

#set gravity in negative z direction
p.setGravity(0, 0, -10)
# p.setRealTimeSimulation(1)

# First let's define a class for the JointInfo.
from dataclasses import dataclass

@dataclass
class Joint:
  index: int
  name: str
  type: int
  gIndex: int
  uIndex: int
  flags: int
  damping: float
  friction: float
  lowerLimit: float
  upperLimit: float
  maxForce: float
  maxVelocity: float
  linkName: str
  axis: tuple
  parentFramePosition: tuple
  parentFrameOrientation: tuple
  parentIndex: int

  def __post_init__(self):
    self.name = str(self.name, 'utf-8')
    self.linkName = str(self.linkName, 'utf-8')

# Let's analyze the R2D2 droid!
print(f"r2d2 unique ID: {r2d2}")
for i in range(p.getNumJoints(r2d2)):
  joint = Joint(*p.getJointInfo(r2d2, i))
  print(joint)


fov, aspect, nearplane, farplane = 60, 1.0, 0.01, 100
projection_matrix = p.computeProjectionMatrixFOV(fov, aspect, nearplane, farplane)


# @numba.jit(nopython=True)
def kuka_camera():
    # Center of mass position and orientation (of link-7)
    com_p, com_o, _, _, _, _ = p.getLinkState(xarm, 6, computeForwardKinematics=True)
    rot_matrix = p.getMatrixFromQuaternion(com_o)
    rot_matrix = np.array(rot_matrix).reshape(3, 3)
    # Initial vectors
    init_camera_vector = (0, 0, 1) # z-axis
    init_up_vector = (0, 1, 0) # y-axis
    # Rotated vectors
    camera_vector = rot_matrix.dot(init_camera_vector)
    up_vector = rot_matrix.dot(init_up_vector)
    view_matrix = p.computeViewMatrix(com_p, com_p + 0.1 * camera_vector, up_vector)
    img = p.getCameraImage(1000, 1000, view_matrix, projection_matrix)
    return [0,0,0]
    
#     img_arr = img
    
#     w = img_arr[0]  #width of the image, in pixels
#     h = img_arr[1]  #height of the image, in pixels
#     rgb = img_arr[2]  #color data RGB
#     dep = img_arr[3]  #depth data
#     print("w=",w,"h=",h)
#     np_img_arr = np.reshape(rgb, (h, w, 4))
#     np_img_arr = np_img_arr * (1. / 255.)
#     return img,rgb,dep
#       return list((0,0,0))


while True:
    p.setRealTimeSimulation(1)
    img,rgb,dep = kuka_camera()
#     cv2.imshow('rgb',rgb)
#     cv2.imshow('dep',dep)
    
p.disconnect()

KeyboardInterrupt: 

In [1]:
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2


import numba
from numba import jit


import pybullet as p
import time
import pybullet_data
from numpngw import write_apng
from IPython.display import Image
import numpy as np

p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())


p.configureDebugVisualizer(p.COV_ENABLE_GUI)
useFixedBase = True
flags = p.URDF_INITIALIZE_SAT_FEATURES

plane_pos = [0,0,-0.625]
plane = p.loadURDF("plane.urdf", plane_pos, flags = flags, useFixedBase=useFixedBase)
table_pos = [0,0,-0.625]
table = p.loadURDF("table/table.urdf", table_pos, flags = flags, useFixedBase=useFixedBase)
xarm = p.loadURDF("meshes/untitiled.urdf", flags = flags, useFixedBase=useFixedBase)

table_pos = [0,1,-0.625]
table2 = p.loadURDF("table/table.urdf", table_pos, flags = flags, useFixedBase=useFixedBase)

sphereRadius = 0.25

p.createCollisionShape(p.GEOM_PLANE)
colBoxId = p.createCollisionShape(p.GEOM_BOX,halfExtents=[sphereRadius, sphereRadius, sphereRadius])

basePosition = [0,0.8,5]
baseOrientation = [0,0,0,1]
mass = 1
visualShapeId = -1

sphereUid = p.createMultiBody(mass, colBoxId, visualShapeId, basePosition,baseOrientation)

#set gravity in negative z direction
p.setGravity(0, 0, -10)
# p.setRealTimeSimulation(1)

# First let's define a class for the JointInfo.
from dataclasses import dataclass

@dataclass
class Joint:
  index: int
  name: str
  type: int
  gIndex: int
  uIndex: int
  flags: int
  damping: float
  friction: float
  lowerLimit: float
  upperLimit: float
  maxForce: float
  maxVelocity: float
  linkName: str
  axis: tuple
  parentFramePosition: tuple
  parentFrameOrientation: tuple
  parentIndex: int

  def __post_init__(self):
    self.name = str(self.name, 'utf-8')
    self.linkName = str(self.linkName, 'utf-8')

# Let's analyze the R2D2 dd!
print(f"r2d2 unique ID: {xarm}")
for i in range(p.getNumJoints(xarm)):
  joint = Joint(*p.getJointInfo(xarm, i))
  print(joint)



r2d2 unique ID: 2
Joint(index=0, name='world_joint', type=4, gIndex=-1, uIndex=-1, flags=0, damping=0.0, friction=0.0, lowerLimit=0.0, upperLimit=-1.0, maxForce=0.0, maxVelocity=0.0, linkName='base_link', axis=(0.0, 0.0, 0.0), parentFramePosition=(0.0, 0.0, 0.0), parentFrameOrientation=(0.0, 0.0, 0.0, 1.0), parentIndex=-1)
Joint(index=1, name='shoulder_pan_joint', type=0, gIndex=7, uIndex=6, flags=1, damping=0.0, friction=0.0, lowerLimit=-6.28318530718, upperLimit=6.28318530718, maxForce=150.0, maxVelocity=3.14, linkName='shoulder_link', axis=(0.0, 0.0, 1.0), parentFramePosition=(0.0, 0.0, 0.1625), parentFrameOrientation=(0.0, 0.0, 1.0, 1.0341155355510722e-13), parentIndex=0)
Joint(index=2, name='shoulder_lift_joint', type=0, gIndex=8, uIndex=7, flags=1, damping=0.0, friction=0.0, lowerLimit=-6.28318530718, upperLimit=6.28318530718, maxForce=150.0, maxVelocity=3.14, linkName='upper_arm_link', axis=(-1.0, 0.0, 4.896583138958022e-12), parentFramePosition=(0.0, 0.0, 0.0), parentFrameOrien